# Creating a Dictionary-based Sentiment Analyzer

In [1]:
import pandas as pd
import nltk
from IPython.display import display
pd.set_option('display.max_columns', None)

### Step 1: Loading in the small_corpus .csv file created in the "creating_dataset" milestone.

In [2]:
reviews = pd.read_csv("../data/small_corpus.csv")

In [3]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"11 30, 2015",A3AC92K59QLYR8,B00503E8S2,ben,Game freezes over and over its unplayable,it just doesn't work,1448841600,NaN,{'Format:': ' Video Game'},NaN
1,1.0,False,"05 19, 2012",A334LHR8DWARY8,B00178630A,Xenocide,I have no problem with needing to be online to...,The only real way to show Blizzard our feeling...,1337385600,23,{'Format:': ' Computer Game'},NaN
2,1.0,True,"10 19, 2014",A28982ODE7ZGVP,B001AWIP7M,Eric Frykberg,NOT GOOD,One Star,1413676800,NaN,{'Format:': ' Video Game'},NaN
3,1.0,True,"09 6, 2015",A19E85RLQCAMI1,B00NASF4MS,Joe,Really not worth the money to buy this game on...,Really not worth the money to buy this game on...,1441497600,2,{'Format:': ' Video Game'},NaN
4,1.0,False,"05 28, 2008",AEMQKS13WC4D2,B00140P9BA,Craig,They need to eliminate the Securom. I purchase...,Securom can ruin a great game,1211932800,55,{'Format:': ' DVD-ROM'},NaN


### Step 2: Tokenizing the sentences and words of the reviews
Here, We're going to test different versions of word tokenizer on reviews. We'll then decide which tokenizer might be better to use.

### Treebank Word Tokenizer

In [4]:
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation
import string

In [5]:
tb_tokenizer = TreebankWordTokenizer()

In [6]:
reviews["rev_text_lower"] = reviews['reviewText'].apply(lambda rev: str(rev)\
                                                        .translate(str.maketrans('', '', punctuation))\
                                                        .replace("<br />", " ")\
                                                        .lower())

In [7]:
reviews[['reviewText','rev_text_lower']].sample(2)

,reviewText,rev_text_lower
2451,Front case of the game was kind of damaged.. B...,front case of the game was kind of damaged but...
2980,"First off, this is a great gaming mouse for th...",first off this is a great gaming mouse for the...


In [8]:
reviews["tb_tokens"] = reviews['rev_text_lower'].apply(lambda rev: tb_tokenizer.tokenize(str(rev)))

In [9]:
pd.set_option('display.max_colwidth', None)

In [10]:
reviews[['reviewText','tb_tokens']].sample(3)

,reviewText,tb_tokens
2355,"First of all I would like to say this game is not nearly as difficult as many people claim. It is very unforgiving of mistakes, but most of the ""trash"" mobs are harder than the bosses.. Many areas/fights you just will die unless you read a guide or a player left a hint beforehand, because there is no other way to know how you need to prepare. The fights are fun, and normally being hit more than once without healing up means you die, but really once you figure out (or look up) what you need to do for the fight it is extremely easy. When people talk about how brutal it is all they really mean is having to fight your way back to the souls you lost or losing them forever if you die again.. not challenging, just unforgiving.\n\nThe tendency system is extremely annoying. To gain pure white character tendency you need to kill black phantoms. To kill black phantoms (other than players) you need pure black world tendency in that area. For that you need to kill yourself several times in body form (and resurrecting with a stone to kill your body again.) The most frustrating part of all this is if you log in the game while online it resets your world tendency, and can reset it randomly while your playing online.. Pure black is a bit easier, just need to kill some friendly npcs. Its like watching paint dry.. why people love doing this is beyond me.\n\nThere is also an area where you literally trudge around in water picking up loot with very little fighting at like 50% reduced walk speed and chug poison dispels since the water randomly poisons you for several hours.. seriously? Granted you can skip the loot and get through it all in under an hour.. but why have an area like that at all?","[first, of, all, i, would, like, to, say, this, game, is, not, nearly, as, difficult, as, many, people, claim, it, is, very, unforgiving, of, mistakes, but, most, of, the, trash, mobs, are, harder, than, the, bosses, many, areasfights, you, just, will, die, unless, you, read, a, guide, or, a, player, left, a, hint, beforehand, because, there, is, no, other, way, to, know, how, you, need, to, prepare, the, fights, are, fun, and, normally, being, hit, more, than, once, without, healing, up, means, you, die, but, really, once, you, figure, out, or, look, up, what, you, need, to, do, for, the, ...]"
3269,awesome!,[awesome]
4006,I would recommend buying this It was just as advertised. Also it came on the time and day that I was told it would come,"[i, would, recommend, buying, this, it, was, just, as, advertised, also, it, came, on, the, time, and, day, that, i, was, told, it, would, come]"


### Casual Tokenizer

In [11]:
from nltk.tokenize.casual import casual_tokenize

In [12]:
reviews['casual_tokens'] = reviews['rev_text_lower'].apply(lambda rev: casual_tokenize(str(rev)))

In [13]:
reviews[['reviewText','casual_tokens','tb_tokens']].sample(3)

,reviewText,casual_tokens,tb_tokens
3430,Beatiful! Create a new look. Excellent. Nice quality. Recommended.,"[beatiful, create, a, new, look, excellent, nice, quality, recommended]","[beatiful, create, a, new, look, excellent, nice, quality, recommended]"
1813,"It was only three chapters in when the game froze during a cut scene and began a high pitched scream for about 30 seconds and then went on like normal. Every cut scene the game decided I didn't want to be holding the weapon I was holding and replaced it with a pistol. Why on earth would I drop a shotgun full of ammo for a pistol with half a clip? Who knows? This isn't the worst game I've ever played and the cut scenes can be enjoyable when they don't look like crap. I would only suggest this to people who enjoy tedious shoot outs, hard boiled detective stories, and don't mind a flawed game. If you can forgive all that's wrong with this game... it might be fun. I for one can't.","[it, was, only, three, chapters, in, when, the, game, froze, during, a, cut, scene, and, began, a, high, pitched, scream, for, about, 30, seconds, and, then, went, on, like, normal, every, cut, scene, the, game, decided, i, didnt, want, to, be, holding, the, weapon, i, was, holding, and, replaced, it, with, a, pistol, why, on, earth, would, i, drop, a, shotgun, full, of, ammo, for, a, pistol, with, half, a, clip, who, knows, this, isnt, the, worst, game, ive, ever, played, and, the, cut, scenes, can, be, enjoyable, when, they, dont, look, like, crap, i, would, only, suggest, this, to, ...]","[it, was, only, three, chapters, in, when, the, game, froze, during, a, cut, scene, and, began, a, high, pitched, scream, for, about, 30, seconds, and, then, went, on, like, normal, every, cut, scene, the, game, decided, i, didnt, want, to, be, holding, the, weapon, i, was, holding, and, replaced, it, with, a, pistol, why, on, earth, would, i, drop, a, shotgun, full, of, ammo, for, a, pistol, with, half, a, clip, who, knows, this, isnt, the, worst, game, ive, ever, played, and, the, cut, scenes, can, be, enjoyable, when, they, dont, look, like, crap, i, would, only, suggest, this, to, ...]"
1340,I have loved every Blizzard game but this one looks to be their first BIG flop. I guess it had to happen sooner or later. They must have gotten to big and are getting dragged down by to many cooks and kitchen(Bad management that used to be great talent but now camp and only bog down the real creatives). It is obvious that dragging out release dates is no longer for a high quality products and is just to cobble together a mesh-mash of yesteryear's leftovers. Granted all they do is re-release the same three games :-P but so far they have been doing great at improving them and adding to the muiltiplayer experience and normally have a good story line to go with it. On a side note I can believe it takes them more than two years to try to make the second part/ expansion of StarCraft II. Diablo 3 is an EXACT copy of Diablo 2. This may sound like a good thing but computers have change a little in the last 10 years and it is just not up to snuff. The story line is also very weak. My advice if you want the feeling of playing Diablo again remember what made it great. It was the first of the MMORPG's it was not quite there but it got people thinking about what could be done and playing together online. This could have been a great blend bringing the classic dungeon game to somewhat massive multiplayer but with only one to a max of 3 other people in your game it is a compleate failure and a relic of the past nothing new to see here move along and find a real MMORPG or a rpg or strategy game that can support at least 8 to 12 players in the same game.,"[i, have, loved, every, blizzard, game, but, this, one, looks, to, be, their, first, big, flop, i, guess, it, had, to, happen, sooner, or, later, they, must, have, gotten, to, big, and, are, getting, dragged, down, by, to, many, cooks, and, kitchenbad, management, that, used, to, be,

### Removing StopWords
This part has been remvoed as removing stop words is not good for sentiment analysis at all!!

In [14]:
#nltk.download('stopwords')

In [15]:
#stop_words = nltk.corpus.stopwords.words('english')

In [16]:
#stop_words.remove("no")

In [17]:
#stop_words.remove("not")

In [18]:
#print(stop_words)

In [19]:
#"not" in stop_words

In [20]:
#len(stop_words)

In [21]:
#from string import punctuation
#print(punctuation)

In [22]:
#reviews['tokens_nosw'] = reviews['tb_tokens'].\
#    apply(lambda words: [w for w in words if w not in stop_words and w not in punctuation and w != ""])

In [23]:
#reviews[['tb_tokens','tokens_nosw']].sample(3)

### Stemming

In [24]:
from nltk.stem.porter import PorterStemmer

In [25]:
stemmer = PorterStemmer()

In [26]:
reviews['tokens_stemmed'] = reviews['tb_tokens'].apply(lambda words: [stemmer.stem(w) for w in words])

In [27]:
reviews[['tb_tokens','tokens_stemmed']].sample(3)

,tb_tokens,tokens_stemmed
1766,"[a, very, buggy, game, with, a, lack, luster, story, mediocre, at, best]","[a, veri, buggi, game, with, a, lack, luster, stori, mediocr, at, best]"
1163,"[this, is, the, ps2, ps3, scenario, repeating, itself, all, over, again, the, same, way, to, the, ps3, will, never, measure, up, to, the, ps2, the, vita, is, milestones, away, from, its, predecessor, the, most, powerful, systems, have, never, been, on, top, through, out, gaming, history, history, repeats, itself, once, again, what, did, the, ps2, have, that, the, ps3, didnt, godlike, 3rd, party, support, but, not, just, that, it, had, the, most, important, aspect, of, all, balance, power, means, nothing, to, a, game, console, games, mean, everything, attracting, people, with, power, instead, of, gameplay, mechanics, and, games, is, the, easy, way, ...]","[thi, is, the, ps2, ps3, scenario, repeat, itself, all, over, again, the, same, way, to, the, ps3, will, never, measur, up, to, the, ps2, the, vita, is, mileston, away, from, it, predecessor, the, most, power, system, have, never, been, on, top, through, out, game, histori, histori, repeat, itself, onc, again, what, did, the, ps2, have, that, the, ps3, didnt, godlik, 3rd, parti, support, but, not, just, that, it, had, the, most, import, aspect, of, all, balanc, power, mean, noth, to, a, game, consol, game, mean, everyth, attract, peopl, with, power, instead, of, gameplay, mechan, and, game, is, the, easi, way, ...]"
4189,[good],[good]


### Lemmatisation

In [28]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [29]:
def penn_to_wn(tag):
    """
        Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [30]:
lemmatizer = WordNetLemmatizer()
def get_lemas(tokens):
    lemmas = []
    for token in tokens:
        pos = penn_to_wn(pos_tag([token])[0][1])
        if pos:
            lemma = lemmatizer.lemmatize(token, pos)
            if lemma:
                lemmas.append(lemma)
    return lemmas

In [31]:
reviews['lemmas'] = reviews['tb_tokens'].apply(lambda tokens: get_lemas(tokens))

In [39]:
reviews[['reviewText','tokens_stemmed','lemmas']].sample(2)

,reviewText,tokens_stemmed,lemmas
4135,Freaking awesome game!!! I did all kinds of side missions that normally I would skip.,"[freak, awesom, game, i, did, all, kind, of, side, mission, that, normal, i, would, skip]","[freak, awesome, game, i, do, kind, side, mission, normally, i, skip]"
1878,"After my 60+ hours of game play, I get more and more frustrated with the game.\n\n+ side: nice graphic, and curve roads are nice.\n\n- side:\nalways online sucks, yes they fixes the logon issue, so I can always logon without waiting. The problems I am having now is that I cannot load some of my cities at all. So many hours of work is wasted\n\nTourism is broken. All the casinos and attractions suddenly lost all the customers for no explainable reasons. I have airport, rail, and ferry running, but all of a sudden people just decide not to visit.\n\nInteractive between cities has lots of issues. Simcity try to be MMO, but they cannot do simple transactions right. Imagine that each city is a character in a MMO, and it takes 10+ minutes for each characters to exchange items/gold? Do you want to play MMO like that?\n\nPublic transports just randomly pick and drop sims, and path finding has lots of issues. Some of the stores say no shoppers and some sims ask where is shopping, even they are right across the street. A $20 game like Cities in Motion does a lot better than Simcity in terms of path finding and transport system.\n\nRendering issues. Some of the buildings keep switching from invisible to visible back and forth. Sometimes when you have a quest, you get zoom in, but what you get is the back of some tall buildings because they are getting int the way. May be they should have an options to change theses behaviors?\n\nThere are many fundamental design issues in this game, and I do not know if EA can or willing to actually fix them.","[after, my, 60, hour, of, game, play, i, get, more, and, more, frustrat, with, the, game, side, nice, graphic, and, curv, road, are, nice, side, alway, onlin, suck, ye, they, fix, the, logon, issu, so, i, can, alway, logon, without, wait, the, problem, i, am, have, now, is, that, i, can, not, load, some, of, my, citi, at, all, so, mani, hour, of, work, is, wast, tourism, is, broken, all, the, casino, and, attract, suddenli, lost, all, the, custom, for, no, explain, reason, i, have, airport, rail, and, ferri, run, but, all, of, a, sudden, peopl, just, decid, not, to, ...]","[hour, game, play, i, get, more, more, frustrate, game, side, nice, graphic, curve, road, be, nice, side, always, online, suck, yes, fix, logon, issue, so, i, always, logon, wait, problem, i, be, have, now, be, i, not, load, city, so, many, hour, work, be, waste, tourism, be, broken, casino, attraction, suddenly, lose, customer, explainable, reason, i, have, airport, rail, ferry, run, sudden, people, just, decide, not, visit, interactive, city, have, lot, issue, simcity, try, be, mmo, not, do, simple, transaction, right, imagine, city, be, character, mmo, take, minute, character, exchange, itemsgold, do, want, play, mmo, public, transport, just, randomly, pick, ...]"


### Sentiment Predictor Baseline Model

In [54]:
def get_sentiment_score(tokens):
    score = 0
    tags = pos_tag(tokens)
    for word, tag in tags:
        wn_tag = penn_to_wn(tag)
        if not wn_tag:
            continue
        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            continue
        
        #most common set:
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        
        score += (swn_synset.pos_score() - swn_synset.neg_score())
        
    return score
                    

In [55]:
## test
swn.senti_synset(wn.synsets("awesome", wn.ADJ)[0].name()).pos_score()

0.875

In [56]:
reviews['sentiment_score'] = reviews['lemmas'].apply(lambda tokens: get_sentiment_score(tokens))

In [57]:
reviews[['reviewText','lemmas','sentiment_score']].sample(5)

,reviewText,lemmas,sentiment_score
3875,It works well just as he said with just a little dirt on the contacts. Check seller ratings to improve the possibility of your old school game working. Ask all your questions,"[work, well, just, say, just, little, dirt, contact, check, seller, rating, improve, possibility, old, school, game, work, ask, question]",0.875
803,I tried to play the game but it would freeze up so I didn't get to play.,"[i, try, play, game, freeze, up, so, i, didnt, get, play]",-0.125
1419,"I cannot see how anyone would like this game. You must have the patience of a python waiting a month to catch prey. My wife bought this game with my PS3 as a gift. Two days later I took it to Gamestop...I was surprised to get 40 buck for it. Well, when it went on sale on Amazon I figured I did not give it a chance so I bought it for PC, well waste of money. It take about an hour before the game starts, and once it does still no action. I cannot even play it anymore.","[i, not, see, anyone, game, have, patience, python, wait, month, catch, prey, wife, bought, game, ps3, gift, day, later, i, take, gamestopi, be, surprised, get, buck, well, go, sale, amazon, i, figure, i, do, not, give, chance, so, i, bought, pc, well, waste, money, take, hour, game, start, once, do, still, action, i, not, even, play, anymore]",-1.125
2861,"True to the brand, lots of good details. Fun skills, upgrades and lots of great missions. I bought for my son but my husband took it over! I love the original songs and score in the game. It can be a little tricky to capture ghosts at first but once you get the hang of it you will find it a lot of fun. I feel the game is a little hard for younger kids so I would not reccomend to kids under 7. I ain't afraid of no ghosts!","[true, brand, lot, good, detail, fun, skill, upgrade, lot, great, mission, i, bought, son, husband, take, i, love, original, song, score, game, be, little, tricky, capture, ghost, first, once, get, hang, find, lot, fun, i, feel, game, be, little, hard, young, kid, so, i, not, reccomend, kid, i, aint, afraid, ghost]",-0.125
3156,Very well made game for being a Wii title. I play on my wii u,"[very, well, make, game, be, wii, title, i, play, wii, u]",0.250


In [58]:
reviews[['reviewText','lemmas','sentiment_score']].sample(5)

reviewText  \
2434  Super Ghouls 'N Ghosts is impossible.  Nah it's not.  It's possible.  It's possible to beat this game.  I know because I've watched people do it before.  But your patience level needs to be at its absolute highest in order to get *anywhere* in this game.  With a game so tough it's no wonder most people back in the day only played it a couple times, realized how hard it was and put it away for years until they eventually sold it.  I know I would've done the same thing had I not had prior knowledge of this games incredible difficulty.\n\nNot getting through level 1 was a bad enough experience for me, but I already knew what to expect going into the game so I didn't mind the loss.  I didn't take losing personally.  No, not when you're playing a game with a reputation for being extremely hard.  You learn to KNOW better!  Some people just didn't know what they were getting into when they played this for the first time (and probably now with a whole new generation of gamers, you could pull a really dirty trick on them and say "Hey!  Check out THIS game!  I know you'll love it!" and see what their reaction is like, lol).  Probably a face-ripping reaction... that being your face.\n\nWhat makes the game so tough is a combination of 3 things- 1 is the play control.  Growing up with the NES generation I never minded awkward moving and jumping if the surrounding danger was kept to a minimum. Unfortunately the surrounding danger in this game is *constant* with many precious deadly surprises around every corner, and enemies are located in places *just* off-screen and appearing magically when you move over one notch.  Just enough for you to take an unavoidable hit in the cheapest way ever.  Throw in water stages and platform logs you have to ride across, and how about a level that moves up and down while riding a platform all the while avoiding surrounding dangerous enemies and their attacks, and that's the kind of nightmare-ish encounters you can expect in Super Ghouls 'N Ghosts.  The game *wants* you, BEGS you, to take a walk into dangerous territory just so you can take a hit.\n\nThe 2nd reason for this games high challenge and for me clearly the biggest reason, is the fact that you can only get once or twice and then you die.  If at least 5 different enemies and attack are occurring all around you giving you little time to react, the generous and actually *right* gameplay method is to allow the player to take at least more than 1 or 2 hits.  Even Castlevania with its amazingly high challenge is courteous enough to allow the player to take at least 3-4 hits.  Imagine if you couldn't even wear protective gear while roaming through the levels of Super Ghouls 'N Ghosts?  Then it simply becomes "1 hit you're dead".  At least Contra allows the player to reappear in the same spot after a death.  No, in this game you either have to start back at the beginning of the stage or the nearest checkpoint.  The gameplay is already sluggish enough- it's not worth the aggravation believe me.\n\nAnd the 3rd reason is one I already mentioned- enemies being cheap.  Too many enemies on screen?  Yes.  Enemies with unpredictable movements with no clear pattern to retaliate?  Oh yeah (those DARN bird creatures have no pattern and you have to deal with more than 1 on screen occasionally!) Oddly the boss fights are relatively easy compared to the regular enemies that often swarm the stages.  Too much commotion taking place to properly focus on enemy elimination is an unforgiveable issue.  I guess the good news is that at least the levels aren't boring.\n\nTo Super Ghouls 'N Ghosts credit a massively challenging game does sometimes have its moments of fun... when you're partying with friends and falling down drunk.  Or when a cat is taking its claws and scratching your face to shreds.  Or when you're so cold your feet feel like swollen icicles.  Or when the Dallas Cowboys are winning.  Under these conditions the game is probably fun, but not whe